In [12]:
import numpy as np

def load_frametimes(base_path, num_cameras=6):
    """
    Load frametimes.npy files for a specified number of cameras.

    Parameters:
    - base_path (str): The base path where the main folder is located.
    - num_cameras (int): The number of cameras to load frametimes for. Default is 6.

    Returns:
    - dict: A dictionary with camera names as keys and frametimes data as values.
    """
    frametimes = {}
    for i in range(1, num_cameras + 1):
        camera_path = f'{base_path}/videos/Camera{i}/frametimes.npy'
        try:
            frametimes[f'Camera{i}'] = np.load(camera_path)
            print(f"Loaded frametimes for Camera{i}")
        except FileNotFoundError:
            print(f"File not found: {camera_path}")
    return frametimes

# Example usage
base_path = '/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_29/20240717_PMCr2'
# base_path = '/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_10_04/20240916V1r2/'
frametimes_data = load_frametimes(base_path)


Loaded frametimes for Camera1
Loaded frametimes for Camera2
Loaded frametimes for Camera3
Loaded frametimes for Camera4
Loaded frametimes for Camera5
Loaded frametimes for Camera6


In [13]:
frametimes_data

{'Camera1': array([[1.00000000e+00, 2.00000000e+00, 3.00000000e+00, ...,
         2.78980000e+04, 2.78990000e+04, 2.79000000e+04],
        [0.00000000e+00, 3.33506080e-02, 6.66986720e-02, ...,
         9.30371212e+02, 9.30404560e+02, 9.30437911e+02]]),
 'Camera2': array([[1.00000000e+00, 2.00000000e+00, 3.00000000e+00, ...,
         2.78980000e+04, 2.78990000e+04, 2.79000000e+04],
        [0.00000000e+00, 3.33473680e-02, 6.66985600e-02, ...,
         9.30369457e+02, 9.30402808e+02, 9.30436159e+02]]),
 'Camera3': array([[1.00000000e+00, 2.00000000e+00, 3.00000000e+00, ...,
         1.57460000e+04, 1.57470000e+04, 1.57480000e+04],
        [0.00000000e+00, 3.33434480e-02, 6.66946720e-02, ...,
         5.41338787e+02, 5.41538886e+02, 5.41705645e+02]]),
 'Camera4': array([[1.00000000e+00, 2.00000000e+00, 3.00000000e+00, ...,
         2.78980000e+04, 2.78990000e+04, 2.79000000e+04],
        [0.00000000e+00, 3.33511680e-02, 6.66946960e-02, ...,
         9.30369634e+02, 9.30402981e+02, 9.30436

In [2]:
def check_max_shapes_consistency(frametimes_data):
    max_shapes = {camera: data.shape[1] for camera, data in frametimes_data.items()}
    print("Maximum shapes for each camera:", max_shapes)
    
    # Get the set of unique shape values
    unique_shapes = set(max_shapes.values())
    
    # Check if all cameras have the same shape
    return len(unique_shapes) == 1

# Example usage:
result = check_max_shapes_consistency(frametimes_data)
print("All cameras have the same shape:", result)


Maximum shapes for each camera: {'Camera1': 27900, 'Camera2': 27900, 'Camera3': 15748, 'Camera4': 27900, 'Camera5': 27900, 'Camera6': 27900}
All cameras have the same shape: False


In [3]:
import numpy as np

def clone_frame_numbers(frametimes_data):
    cloned_frame_numbers = {}
    for camera, data in frametimes_data.items():
        # Clone the frame numbers (data[0]) for each camera
        cloned_frame_numbers[camera] = np.copy(data[0])
    return cloned_frame_numbers

# Usage
cloned_frame_numbers = clone_frame_numbers(frametimes_data)


In [4]:
import numpy as np

def create_standard_timeline(frametimes_data, fps=30):
    # Extract the end times and frame numbers for each camera
    end_times = {camera: frametimes_data[camera][1][-1] for camera in frametimes_data.keys()}
    max_frame_numbers = {camera: frametimes_data[camera][0][-1] for camera in frametimes_data.keys()}
    
    # Determine the maximum end time and frame number across all cameras
    max_end_time = int(round(max(end_times.values())))
    max_frame_number = max(max_frame_numbers.values())
    
    # Calculate the interval between frames
    frame_interval = 1 / fps
    
    # Generate standard timeline and frame numbers
    standard_timeline = np.arange(0, max_end_time, frame_interval)
    standard_frame_numbers = np.arange(0, max_frame_number + 1)
    
    return standard_frame_numbers, standard_timeline

standard_timeline = create_standard_timeline(frametimes_data)

In [19]:
frametimes_data = {
    'Camera1': np.array([
        np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),       # Frame numbers
        np.array([0, 0.033, 0.067, 0.1, 0.133, 0.167, 0.2, 0.233, 0.267, 0.3])  # Timestamps
    ]),
    'Camera5': np.array([
        np.array([0, 1, 2, 4, 5, 6, 7, 9]),             # Frame numbers, with 3 and 8 dropped
        np.array([0, 0.033, 0.067, 0.133, 0.167, 0.2, 0.233, 0.3])  # Timestamps
    ])
}

# Modified align_to_standard_timeline to align both timestamps and frame numbers
def align_to_standard_timeline(frametimes_data, standard_frame_numbers, standard_timeline):
    aligned_data = {}
    
    for camera, data in frametimes_data.items():
        frame_numbers, timestamps = data[0], data[1]
        aligned_frames = []
        aligned_timestamps = []
        last_frame_number = frame_numbers[0]  # Start with the first frame number as the prior value

        for frame in standard_frame_numbers:
            idx = np.searchsorted(frame_numbers, frame, side="left")
            if idx < len(frame_numbers) and frame_numbers[idx] == frame:
                # Frame matches; use it and update last_frame_number
                aligned_frames.append(frame_numbers[idx])
                aligned_timestamps.append(timestamps[idx])
                last_frame_number = frame_numbers[idx]
            else:
                # Frame is missing, fill with the previous frame number and NaN timestamp
                aligned_frames.append(last_frame_number)
                aligned_timestamps.append(np.nan)

        aligned_data[camera] = {
            "frames": np.array(aligned_frames),
            "timestamps": np.array(aligned_timestamps)
        }
        print(f"Aligned data for {camera} with {np.isnan(aligned_timestamps).sum()} dropped frames filled.")
    
    return aligned_data

# Main function to process camera data
def process_camera_data(frametimes_data, fps=30):
    # Step 1: Check for consistency in frame shapes
    # if not check_max_shapes_consistency(frametimes_data):
    #     raise ValueError("Cameras have inconsistent shapes.")
    
    # Step 2: Create the standard timeline and frame numbers
    standard_frame_numbers, standard_timeline = create_standard_timeline(frametimes_data, fps)
    
    # Step 3: Align camera data to the standard timeline
    aligned_frametimes = align_to_standard_timeline(frametimes_data, standard_frame_numbers, standard_timeline)
    
    print("All cameras aligned successfully.")
    return aligned_frametimes, standard_frame_numbers, standard_timeline

# Example usage
aligned_frametimes, standard_frame_numbers, standard_timeline = process_camera_data(frametimes_data, fps=30)


# Print aligned frame numbers
for camera, frames in aligned_frametimes.items():
    print(f"{camera} aligned frame numbers:", frames)

Aligned data for Camera1 with 0 dropped frames filled.
Aligned data for Camera5 with 2 dropped frames filled.
All cameras aligned successfully.
Camera1 aligned frame numbers: {'frames': array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]), 'timestamps': array([0.   , 0.033, 0.067, 0.1  , 0.133, 0.167, 0.2  , 0.233, 0.267,
       0.3  ])}
Camera5 aligned frame numbers: {'frames': array([0., 1., 2., 2., 4., 5., 6., 7., 7., 9.]), 'timestamps': array([0.   , 0.033, 0.067,   nan, 0.133, 0.167, 0.2  , 0.233,   nan,
       0.3  ])}


In [17]:
aligned_frametimes

{'Camera1': array([0.   , 0.033, 0.067, 0.1  , 0.133, 0.167, 0.2  , 0.233, 0.267,
        0.3  ]),
 'Camera5': array([0.   , 0.033, 0.067,   nan, 0.133, 0.167, 0.2  , 0.233,   nan,
        0.3  ])}

In [8]:
base_path = '/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ/2024_08_29/20240717_PMCr2'
frametimes_data = load_frametimes(base_path)
cloned_frame_numbers = clone_frame_numbers(frametimes_data)
standard_frame_numbers, standard_timeline = create_standard_timeline(frametimes_data)
aligned_frametimes = align_to_standard_timeline(frametimes_data, standard_frame_numbers)

Camera1 aligned frame numbers: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Camera5 aligned frame numbers: [0, 1, 2, 2, 2, 2, 2, 2, 2]


In [9]:
# Align each camera's frame data to the standard timeline
def align_to_standard_timeline(frametimes_data, standard_frame_numbers):
    aligned_frametimes = {}
    
    for camera, data in frametimes_data.items():
        camera_frame_numbers = data[0]
        aligned_frames = []

        for frame in standard_frame_numbers:
            idx = np.searchsorted(camera_frame_numbers, frame, side="left")
            if idx < len(camera_frame_numbers) and camera_frame_numbers[idx] == frame:
                aligned_frames.append(data[1][idx])
            else:
                aligned_frames.append(np.nan)

        aligned_frametimes[camera] = np.array(aligned_frames)
        print(f"Aligned data for {camera} with {np.isnan(aligned_frames).sum()} dropped frames filled.")
    
    return aligned_frametimes

In [16]:
# Access frame numbers and timestamps for Camera1
camera1_frame_numbers = frametimes_data['Camera1'][0]  # Frame numbers
camera1_timestamps = frametimes_data['Camera1'][1]     # Timestamps

print("Camera1 - Frame Numbers:\n", camera1_frame_numbers)
print("Camera1 - Timestamps:\n", camera1_timestamps)


Camera1 - Frame Numbers:
 [1.0000e+00 2.0000e+00 3.0000e+00 ... 2.7898e+04 2.7899e+04 2.7900e+04]
Camera1 - Timestamps:
 [0.00000000e+00 3.33506080e-02 6.66986720e-02 ... 9.30371212e+02
 9.30404560e+02 9.30437911e+02]


In [17]:
frametimes_data['Camera1'][0][3]

4.0

In [21]:
frametimes_data['Camera1'][1][-1]

930.4379109120001

In [40]:
end_times = {camera: frametimes_data[camera][1][-1] for camera in frametimes_data.keys()}
max_frame_numbers = {camera: frametimes_data[camera][0][-1] for camera in frametimes_data.keys()}


In [23]:
end_times

{'Camera1': 930.4379109120001,
 'Camera2': 930.436158824,
 'Camera3': 541.7056454640001,
 'Camera4': 930.436328672,
 'Camera5': 930.4349932160001,
 'Camera6': 930.436672144}

In [45]:
# max_end_time = max(end_times.values())
# max_end_time = round(max(end_times.values()), 3)  # Round to 3 decimal places, for example
max_end_time = int(round(max(end_times.values())))
max_frame_number = max(max_frame_numbers.values())



In [46]:
max_end_time

930

In [44]:
max_frame_number

27900.0

In [50]:
import numpy as np

frame_interval = 1 / 30  # Interval between frames at 30 fps
standard_timeline = np.arange(0, max_end_time, frame_interval)
standard_frame_numbers = np.arange(0, max_frame_number + 1)


In [48]:
standard_timeline

array([0.00000000e+00, 3.33333333e-02, 6.66666667e-02, ...,
       9.29900000e+02, 9.29933333e+02, 9.29966667e+02])

In [53]:
standard_frame_numbers

array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 2.7898e+04, 2.7899e+04,
       2.7900e+04])

In [52]:
standard_frame_numbers[-1]

27900.0